In [1]:
import boto3
fraudDetector = boto3.client('frauddetector')
            
response = fraudDetector.get_detectors()
print(response)


{'detectors': [], 'ResponseMetadata': {'RequestId': '53dbccad-e6a3-4632-8fca-ee74132aa078', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 05:20:53 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '16', 'connection': 'keep-alive', 'x-amzn-requestid': '53dbccad-e6a3-4632-8fca-ee74132aa078'}, 'RetryAttempts': 0}}


In [3]:
# create event


import boto3
fraudDetector = boto3.client('frauddetector')

 #Create variable email_address
fraudDetector.create_variable(
name = 'email_address',
variableType = 'EMAIL_ADDRESS',
dataSource = 'EVENT',
dataType = 'STRING',
defaultValue = '<unknown>'
)

#Create variable ip_address
fraudDetector.create_variable(
name = 'ip_address',
variableType = 'IP_ADDRESS',
dataSource = 'EVENT',
dataType = 'STRING',
defaultValue = '<unknown>'
)


{'ResponseMetadata': {'RequestId': '173ba337-2b93-4c91-9697-05e51cd6b8d6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 05:24:57 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '173ba337-2b93-4c91-9697-05e51cd6b8d6'},
  'RetryAttempts': 0}}

In [4]:
# create entity


import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.put_entity_type(
name = 'sample_customer',
description = 'sample customer entity type'
)


{'ResponseMetadata': {'RequestId': '5906b087-8a63-43a4-8d71-97dbcd12507c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 05:25:41 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5906b087-8a63-43a4-8d71-97dbcd12507c'},
  'RetryAttempts': 0}}

In [5]:
# create label


import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.put_label(
name = 'fraud',
description = 'label for fraud events'
)

fraudDetector.put_label(
name = 'legit',
description = 'label for legitimate events'
)


{'ResponseMetadata': {'RequestId': '78cb8ec0-5b25-4df2-a550-e1f0a03651ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 05:26:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '78cb8ec0-5b25-4df2-a550-e1f0a03651ef'},
  'RetryAttempts': 0}}

In [6]:
# create event type


import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.put_event_type (
name = 'sample_registration',
eventVariables = ['ip_address', 'email_address'],
labels = ['legit', 'fraud'],
entityTypes = ['sample_customer'])


{'ResponseMetadata': {'RequestId': 'b8642743-4cda-4fe6-ab6a-8b7034137903',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 05:27:37 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b8642743-4cda-4fe6-ab6a-8b7034137903'},
  'RetryAttempts': 0}}

In [7]:
# building a model

import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.create_model (
modelId = 'sample_fraud_detection_model',
eventTypeName = 'sample_registration',
modelType = 'ONLINE_FRAUD_INSIGHTS')



{'ResponseMetadata': {'RequestId': '0667d07c-189d-4a08-88a6-cb304e0fa8c9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 05:29:16 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0667d07c-189d-4a08-88a6-cb304e0fa8c9'},
  'RetryAttempts': 0}}

In [ ]:
# if creating arn role a create it from fraud console...from "Models" section
# after this training will start...it takes some time....

In [18]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.create_model_version (
modelId = 'sample_fraud_detection_model',
modelType = 'ONLINE_FRAUD_INSIGHTS',
trainingDataSource = 'EXTERNAL_EVENTS',
trainingDataSchema = {
    'modelVariables' : ['ip_address', 'email_address'],
    'labelSchema' : {
        'labelMapper' : {
            'FRAUD' : ['fraud'],
            'LEGIT' : ['legit']
        }
    }
}, 
externalEventsDetail = {
    'dataLocation' : 's3://ml-flow01/registration_data_20K_minimum.csv',
    'dataAccessRoleArn' : 'arn:aws:iam::024615928572:role/service-role/AmazonFraudDetector-DataAccessRole-1617170865106'
}
)


{'modelId': 'sample_fraud_detection_model',
 'modelType': 'ONLINE_FRAUD_INSIGHTS',
 'modelVersionNumber': '1.0',
 'status': 'TRAINING_IN_PROGRESS',
 'ResponseMetadata': {'RequestId': '895880ad-9361-4b8d-9c77-809b320419a2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 06:08:20 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '137',
   'connection': 'keep-alive',
   'x-amzn-requestid': '895880ad-9361-4b8d-9c77-809b320419a2'},
  'RetryAttempts': 0}}

In [21]:
# after approx 2 hr...

In [22]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.update_model_version_status (
modelId = 'sample_fraud_detection_model',
modelType = 'ONLINE_FRAUD_INSIGHTS',
modelVersionNumber = '1.00',
status = 'ACTIVE'
)


{'ResponseMetadata': {'RequestId': '332bda49-4166-409e-913b-176d1fa9f259',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 07:50:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '332bda49-4166-409e-913b-176d1fa9f259'},
  'RetryAttempts': 0}}

In [23]:
# create a detector

In [24]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.put_detector (
detectorId = 'sample_detector',
eventTypeName = 'sample_registration'
)


{'ResponseMetadata': {'RequestId': '3ce01bff-8245-4806-b708-28dbfd1f09b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 07:51:27 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3ce01bff-8245-4806-b708-28dbfd1f09b0'},
  'RetryAttempts': 0}}

In [25]:
# create an outcome

In [26]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.put_outcome(
name = 'verify_customer',
description = 'this outcome initiates a verification workflow'
)

fraudDetector.put_outcome(
name = 'review',
description = 'this outcome sidelines event for review'
)

fraudDetector.put_outcome(
name = 'approve',
description = 'this outcome approves the event'
)


{'ResponseMetadata': {'RequestId': '36d46fe1-0ec8-4dbd-84f2-e63c9fa30744',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 07:51:53 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '36d46fe1-0ec8-4dbd-84f2-e63c9fa30744'},
  'RetryAttempts': 0}}

In [36]:
# create a rule...
# it creates three different rules 
# (high_fraud_risk, medium_fraud_risk, and low_fraud_risk)


In [31]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.create_rule(
ruleId = 'high_fraud_risk',
detectorId = 'sample_detector',
expression = '$sample_fraud_detection_model_insightscore > 900',
language = 'DETECTORPL',
outcomes = ['verify_customer']
)

fraudDetector.create_rule(
ruleId = 'medium_fraud_risk',
detectorId = 'sample_detector',
expression = '$sample_fraud_detection_model_insightscore <= 900 and $sample_fraud_detection_model_insightscore > 700',
language = 'DETECTORPL',
outcomes = ['review']
)

fraudDetector.create_rule(
ruleId = 'low_fraud_risk',
detectorId = 'sample_detector',
expression = '$sample_fraud_detection_model_insightscore <= 700',
language = 'DETECTORPL',
outcomes = ['approve']
)


{'rule': {'detectorId': 'sample_detector',
  'ruleId': 'low_fraud_risk',
  'ruleVersion': '1'},
 'ResponseMetadata': {'RequestId': 'f543f6dd-7ed7-480c-8d9a-add8a93416df',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 08:01:10 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f543f6dd-7ed7-480c-8d9a-add8a93416df'},
  'RetryAttempts': 0}}

In [27]:
# create a detector version

In [32]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.create_detector_version(
detectorId = 'sample_detector',
rules = [{
    'detectorId' : 'sample_detector',
    'ruleId' : 'high_fraud_risk',
    'ruleVersion' : '1'
},
{
    'detectorId' : 'sample_detector',
    'ruleId' : 'medium_fraud_risk',
    'ruleVersion' : '1'
},
{
    'detectorId' : 'sample_detector',
    'ruleId' : 'low_fraud_risk',
    'ruleVersion' : '1'
}
],
modelVersions = [{
    'modelId' : 'sample_fraud_detection_model',
    'modelType': 'ONLINE_FRAUD_INSIGHTS',
    'modelVersionNumber' : '1.00'
}],
ruleExecutionMode = 'FIRST_MATCHED'
)


{'detectorId': 'sample_detector',
 'detectorVersionId': '1',
 'status': 'DRAFT',
 'ResponseMetadata': {'RequestId': 'f7d89d49-60c8-42a9-bc95-2e02c173f9fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 08:01:17 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '73',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f7d89d49-60c8-42a9-bc95-2e02c173f9fa'},
  'RetryAttempts': 0}}

In [34]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.update_detector_version_status(
detectorId = 'sample_detector',
detectorVersionId = '1',
status = 'ACTIVE'
)


{'ResponseMetadata': {'RequestId': '77da8418-b19c-4074-9715-bd995061d730',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 08:01:44 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '77da8418-b19c-4074-9715-bd995061d730'},
  'RetryAttempts': 0}}

In [29]:
# get fraud prediction

In [35]:
import boto3
fraudDetector = boto3.client('frauddetector')

fraudDetector.get_event_prediction(
detectorId = 'sample_detector',
eventId = '802454d3-f7d8-482d-97e8-c4b6db9a0428',
eventTypeName = 'sample_registration',
eventTimestamp = '2020-07-13T23:18:21Z',
entities = [{'entityType':'sample_customer', 'entityId':'12345'}],
eventVariables = {
    'email_address' : 'johndoe@exampledomain.com',
    'ip_address' : '1.2.3.4'
}
)


{'modelScores': [{'modelVersion': {'modelId': 'sample_fraud_detection_model',
    'modelType': 'ONLINE_FRAUD_INSIGHTS',
    'modelVersionNumber': '1.0'},
   'scores': {'sample_fraud_detection_model_insightscore': 967.0}}],
 'ruleResults': [{'ruleId': 'high_fraud_risk',
   'outcomes': ['verify_customer']}],
 'ResponseMetadata': {'RequestId': 'bf285ac5-8d74-4b2a-9336-fa1fa69262ac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 31 Mar 2021 08:01:50 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '277',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bf285ac5-8d74-4b2a-9336-fa1fa69262ac'},
  'RetryAttempts': 0}}